# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [8]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
print(data.head())
print(data.shape)

  id ad  education  ofjobs  yearsexp  honors  volunteer  military  empholes  \
0  b  1          4       2         6       0          0         0         1   
1  b  1          3       3         6       0          1         1         0   
2  b  1          4       1         6       0          0         0         0   
3  b  1          3       4         6       0          1         0         1   
4  b  1          3       3        22       0          0         0         0   

   occupspecific    ...      compreq  orgreq  manuf  transcom  bankreal trade  \
0             17    ...          1.0     0.0    1.0       0.0       0.0   0.0   
1            316    ...          1.0     0.0    1.0       0.0       0.0   0.0   
2             19    ...          1.0     0.0    1.0       0.0       0.0   0.0   
3            313    ...          1.0     0.0    1.0       0.0       0.0   0.0   
4            313    ...          1.0     1.0    0.0       0.0       0.0   0.0   

  busservice othservice  missind  owne

Q1:

We could use A/B test. The central limit theorem (CLT) is a statistical theory that states that given a sufficiently large sample size from a population with a finite level of variance, the mean of all samples from the same population will be approximately equal to the mean of the population. Furthermore, all of the samples will follow an approximate normal distribution pattern, with all variances being approximately equal to the variance of the population divided by each sample's size.

In our data, there are 4870 records in total (a large sample), and therefore we could say that CLT applies.

In [65]:
#separate the dataset and see the fraction for each first
df = data[['race','call']]
df_b = df[df.race=='b']['call']
df_w = df[df.race=='w']['call']
frac_b = sum(df_b)/len(df_b)
frac_w = sum(df_w)/len(df_w)
print(frac_b)
print(frac_w)

0.06447638603696099
0.09650924024640657


Q2:

* Null hypothesis: race (b/w) has no impact on the call-back rate (the difference is purely due to chance)
* Alternative hypothesis: race has impact on the call-back rate 

In [72]:
#Q3
#first we define a fraction difference function 
def frac_diff(data1, data2):
    """Compute the difference in fraction of two data frames for column 'col'"""
    frac_diff12 = sum(data1) / len(data1) - sum(data2) / len(data2) 
    return frac_diff12

In [73]:
#define functions to draw permutation replicates
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""
    data = np.concatenate((data1.values,data2.values)) 
    permuted_data = np.random.permutation(data)

    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

In [74]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    perm_replicates = np.empty(size)

    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

In [80]:
#point estimate
obs_frac_diff = frac_diff(df_b, df_w)
print(obs_frac_diff)

-0.032032854209445585


In [81]:
#actual testing part
perm_replicates = draw_perm_reps(df_b, df_w, frac_diff, 10000)

# Compute and print p-value: p
p = np.sum(perm_replicates <= obs_frac_diff) / len(perm_replicates)
print('p-value =', format(p, '.6f'))

('p-value =', '0.000000')


In [85]:
#double-check using statsmodel z-test as well
from statsmodels.stats.weightstats import ztest
z_score, pValue = ztest(x1= df_b, x2=df_w, value=0, alternative='two-sided', usevar='pooled', ddof=1)
print ("z_score:", z_score, 'p-value:',pValue)

('z_score:', -4.114705356750735, 'p-value:', 3.8767429116085706e-05)


Q4:

The p-value of 0.000000 from bootstrap permutation test and of 3.8767429116085706e-05 from z-test certainly rejects the null hypothesis of race has no impact on the call back rate. In other words, there is very unlikely that difference in call-back rate in two races is due to chance.

Q5:

My analysis only indicates that race has a statistically significant impact on the call-back rate. As I haven't investigated other information in the data, such as columns of 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer' etc., it's hard to conclude that race is the 'most important' factor in the call-back success. In order to make inferences on which factor is the most important, I would need to examine other factors(columns) as well. Possible tests that could be performed include: Chi-Square test, logistic regression to evaluate other factor's importance.